In [1]:
#Importing libraries

import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 
import requests

print("Libraries imported successfully......")


Libraries imported successfully......


# i. Player data Current Season


In [2]:
### Function to return player url 
def format_player_url(player_id):
    # Define the base URL without curly brackets
    base_url = 'https://fantasy.premierleague.com/api/element-summary/{}/'
    
    # Replace the placeholder '{}' with the actual player_id
    formatted_url = base_url.format(player_id)
    
    return formatted_url

# Example usage:
player_id = 447  # Replace this with the actual player ID
formatted_url = format_player_url(player_id)
print(formatted_url)


https://fantasy.premierleague.com/api/element-summary/447/


In [3]:
#This cell imports individual player information
r = requests.get(format_player_url(308))
player_json = r.json()
player_fixtures_df = pd.DataFrame(player_json['fixtures'])
player_history_df = pd.DataFrame(player_json['history'])
player_history_past_df = pd.DataFrame(player_json['history_past'])



In [4]:
# Imported full data from last seasons

url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
json = r.json()


#Convert to df

elements_df = pd.DataFrame(json['elements'])

In [5]:
elements = elements_df.loc[:,['id','team','web_name','first_name','second_name']]
elements = elements.rename(columns = {'id': 'player_id', 'team':'team_id'})

In [6]:
# List of columns to drop
columns_to_drop = ['was_home', 'bps', 'kickoff_time','influence', 'creativity',
       'threat', 'ict_index', 'starts', 'expected_goals', 'expected_assists',
       'expected_goal_involvements', 'expected_goals_conceded', 'value',
       'transfers_balance', 'selected', 'transfers_in', 'transfers_out','fixture', 'team_h_score',
       'team_a_score']

# Use the drop method to remove the specified columns
player_history_df.drop(columns=columns_to_drop, inplace=True)


In [7]:
player = player_history_df.rename(columns = {'element': 'player_id', 'round': 'event_id'})


In [8]:
filtered_teams = pd.read_csv('filtered_teams.csv')

In [9]:
Haaland = elements[elements['second_name']=='Haaland']

In [10]:
Haaland

,player_id,team_id,web_name,first_name,second_name
447,355,13,Haaland,Erling,Haaland


In [11]:
player =  player.merge(elements, on = 'player_id')
player = player.merge(filtered_teams, on = ['team_id','event_id'])
#player = player.drop_duplicates(subset='event_id')


# ii. Analysis

In [12]:
filtered_teams.columns

Index(['team_id', 'team_name', 'team_short_name', 'unavailable', 'event_id',
       'fixture_id', 'fixture_difficulty', 'is_home', 'kickoff_time',
       'started', 'Attack_Strength', 'team_score', 'Form', 'Numerical_Form',
       'Win_percentage', 'Strength', 'Defence_Strength', 'Home_Form',
       'Home_Numerical_Form', 'Home_Win_percentage', 'Home_Strength',
       'Home_Defence_Strength', 'Home_Attack_Strength', 'Away_Form',
       'Away_Numerical_Form', 'Away_Win_percentage', 'Away_Strength',
       'Away_Defence_Strength', 'Away_Attack_Strength'],
      dtype='object')

In [13]:
player

,player_id,opponent_team,total_points,event_id,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,...,Home_Win_percentage,Home_Strength,Home_Defence_Strength,Home_Attack_Strength,Away_Form,Away_Numerical_Form,Away_Win_percentage,Away_Strength,Away_Defence_Strength,Away_Attack_Strength
0,308,7,5,1,76,0,1,0,1,0,...,NaN,NaN,NaN,NaN,D,0.512967,0.000000,0.170941,0.184439,0.272977
1,308,3,5,2,85,1,0,0,1,0,...,100.0,0.874929,0.873992,0.783229,NaN,NaN,NaN,NaN,NaN,NaN
2,308,15,5,3,90,0,1,0,1,0,...,NaN,NaN,NaN,NaN,DW,0.673846,50.000000,0.567725,0.571423,0.609901
3,308,2,8,4,90,1,0,1,0,0,...,100.0,0.927800,0.929055,0.871649,NaN,NaN,NaN,NaN,NaN,NaN
4,308,20,10,5,90,0,2,0,1,0,...,NaN,NaN,NaN,NaN,DWW,0.766154,66.666667,0.697768,0.697946,0.720376
5,308,19,10,6,90,1,0,0,1,0,...,100.0,0.960773,0.961943,0.930595,NaN,NaN,NaN,NaN,NaN,NaN
6,308,18,1,7,73,0,0,0,1,0,...,NaN,NaN,NaN,NaN,DWWL,0.522198,50.000000,0.515735,0.513070,0.530827
7,308,5,15,8,90,2,0,0,2,0,...,NaN,NaN,NaN,NaN,DWWLD,0.520352,40.000000,0.446777,0.446035,0.481458


In [14]:
#Features_engineered = ['Minutes_per_game', 'Player_Strength', 'XA', 'XG', 'XS']
features = ['fixture_difficulty','kickoff_time', 'started',
       'is_home', 'Attack_Strength', 'team_score', 'Form', 'Numerical_Form',
       'Win_percentage', 'Strength', 'Defence_Strength', 'Home_Form',
       'Home_Numerical_Form', 'Home_Win_percentage', 'Home_Strength',
       'Home_Defence_Strength', 'Home_Attack_Strength', 'Away_Form',
       'Away_Numerical_Form', 'Away_Win_percentage', 'Away_Strength',
       'Away_Defence_Strength', 'Away_Attack_Strength']
labels = ['goals_scored', 'assists', 'clean_sheets', 'goals_conceded', 'saves']

In [15]:
player['kickoff_time'] = pd.to_datetime(player['kickoff_time'])



In [16]:
player.set_index('kickoff_time', inplace=True)


In [17]:
player = player[player['started']==True]


In [18]:
player#.loc[:,['started','event_id']]

,player_id,opponent_team,total_points,event_id,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,...,Home_Win_percentage,Home_Strength,Home_Defence_Strength,Home_Attack_Strength,Away_Form,Away_Numerical_Form,Away_Win_percentage,Away_Strength,Away_Defence_Strength,Away_Attack_Strength
kickoff_time,,,,,,,,,,,,,,,,,,,,,
2023-08-13 15:30:00+00:00,308,7,5,1,76,0,1,0,1,0,...,NaN,NaN,NaN,NaN,D,0.512967,0.000000,0.170941,0.184439,0.272977
2023-08-19 14:00:00+00:00,308,3,5,2,85,1,0,0,1,0,...,100.0,0.874929,0.873992,0.783229,NaN,NaN,NaN,NaN,NaN,NaN
2023-08-27 15:30:00+00:00,308,15,5,3,90,0,1,0,1,0,...,NaN,NaN,NaN,NaN,DW,0.673846,50.000000,0.567725,0.571423,0.609901
2023-09-03 13:00:00+00:00,308,2,8,4,90,1,0,1,0,0,...,100.0,0.927800,0.929055,0.871649,NaN,NaN,NaN,NaN,NaN,NaN
2023-09-16 11:30:00+00:00,308,20,10,5,90,0,2,0,1,0,...,NaN,NaN,NaN,NaN,DWW,0.766154,66.666667,0.697768,0.697946,0.720376
2023-09-24 13:00:00+00:00,308,19,10,6,90,1,0,0,1,0,...,100.0,0.960773,0.961943,0.930595,NaN,NaN,NaN,NaN,NaN,NaN
2023-09-30 16:30:00+00:00,308,18,1,7,73,0,0,0,1,0,...,NaN,NaN,NaN,NaN,DWWL,0.522198,50.000000,0.515735,0.513070,0.530827
2023-10-08 13:00:00+00:00,308,5,15,8,90,2,0,0,2,0,...,NaN,NaN,NaN,NaN,DWWLD,0.520352,40.000000,0.446777,0.446035,0.481458


In [19]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Load your data into a Pandas DataFrame (Assuming you have already processed the data)
# Feature engineering code is the same as in the previous response

# Select features and labels
features = ['fixture_difficulty', 'started', 'is_home', 'Attack_Strength', 'team_score', 'Numerical_Form', 'Win_percentage', 'Strength', 'Defence_Strength']
labels = ['goals_scored', 'assists', 'clean_sheets', 'goals_conceded', 'saves']

# Choose a label to predict
label_to_predict = 'goals_scored'

# Select the features and label you want to use
X = player[features]
y = player[label_to_predict]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features (recommended for neural networks)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build a TensorFlow model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer with 1 neuron for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f"Mean Squared Error on the test set: {loss:.2f}")

# Make predictions
predictions = model.predict(X_test)

# You can use the predictions for further analysis or visualization


Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 0.9201 - val_loss: 0.5168
Epoch 2/100
1/1 [==============================] - 0s 267ms/step - loss: 0.8699 - val_loss: 0.4827
Epoch 3/100
1/1 [==============================] - 0s 193ms/step - loss: 0.8241 - val_loss: 0.4499
Epoch 4/100
1/1 [==============================] - 0s 177ms/step - loss: 0.7833 - val_loss: 0.4184
Epoch 5/100
1/1 [==============================] - 0s 186ms/step - loss: 0.7468 - val_loss: 0.3883
Epoch 6/100
1/1 [==============================] - 0s 88ms/step - loss: 0.7148 - val_loss: 0.3597
Epoch 7/100
1/1 [==============================] - 0s 68ms/step - loss: 0.6873 - val_loss: 0.3329
Epoch 8/100
1/1 [==============================] - 0s 68ms/step - loss: 0.6631 - val_loss: 0.3074
Epoch 9/100
1/1 [==============================] - 0s 68ms/step - loss: 0.6433 - val_loss: 0.2843
Epoch 10/100
1/1 [==============================] - 0s 68ms/step - loss: 0.6274 - val_loss: 0.2631
Epoch 11/100
1/1 

In [20]:
predictions

array([[0.89263284],
       [0.21047986]], dtype=float32)